In [1]:
from pathlib import Path
import numpy as np
from dataclasses import dataclass
import re
import math

In [171]:
@dataclass
class point:
    x : int
    y : int

    @property
    def diagonal(self):
        return self.x+self.y

    @property
    def offset(self):
        return self.y-self.x

    @classmethod
    def from_do(cls, diagonal, offset):
        x = ((diagonal - offset)/2)
        y = ((diagonal + offset)/2)
        pt = cls(x,y)
        assert(pt.offset == offset) and (pt.diagonal == diagonal)
        return pt

    def __hash__(self):
        return hash((self.x, self.y))

In [172]:
@dataclass
class interval:
    c1 : int
    c2 : int

    @property
    def non_negative(self) -> bool :
        return (self.c2 >= self.c1)

In [173]:
def to_interval(s : point, b : point, y : int):
    size = abs(s.x - b.x) + abs(s.y- b.y)
    length = size - abs(s.y - y)
    return interval(s.x-length, s.x+length)

In [222]:
allowed = '0123456789-'
def parse_line(line):

    sx,sy,bx,by = map(int,(''.join([l if l in allowed else ' ' for l in line])).split())
    return point(sx,sy), point(bx,by)

In [223]:
def read(prefix='data'):
    lines = Path(f'{prefix}/15.txt').read_text().rstrip().split('\n')
    return [parse_line(line) for line in lines]

In [224]:
def row_length(lines, y, display=False):
    S = set()
    for (s,b) in lines:
        ivl = to_interval(s,b, y)
        if ivl.non_negative:
            for x in range(ivl.c1, ivl.c2+1, 1):
                S.add(x)

    for (_,b) in lines:
        if b.y == y:
            if b.x in S:
                S.remove(b.x)

    return S

In [225]:
def show(S):
    return len(S), ''.join(['#' if i in S else '.' for i in range(min(S), len(S)-min(S))])

In [226]:
test = read('test')
show(row_length(test, y=10))

(26, '####.######################...')

In [227]:
lines = read()
len(row_length(lines, y=2000000))


5461729

In [228]:
@dataclass
class Dline:
    x : int
    y : int
    length : int

    def __iter__(self):
        for i in range(self.length):
            yield point(self.x + i, self.y - i)

In [229]:
[x for x in Dline(1,2,3)]

[point(x=1, y=2), point(x=2, y=1), point(x=3, y=0)]

In [230]:
@dataclass
class Diamond:
    x : int
    y : int
    size : int

    @classmethod
    def from_sensorbeacon(cls,s,b):
        size = abs(s.x - b.x) + abs(s.y- b.y)
        return cls(s.x,s.y, size)

    @property
    def top(self):
        return point(self.x, self.y - self.size)

    @property
    def bottom(self):
        return point(self.x, self.y + self.size)

    @property
    def left(self):
        return point(self.x - self.size, self.y)

    @property
    def right(self):
        return point(self.x + self.size, self.y)

    @property
    def corners(self):
        return (self.top, self.bottom, self.left, self.right)

    def contains(self, pt : point):
        dist = abs(self.x - pt.x) + abs(self.y - pt.y)
        return dist <= self.size

    def line_on(self, diagonal):
        if (self.bottom.diagonal < diagonal) or (self.left.diagonal > diagonal):
            return Dline(0,0,0)
        offset = diagonal - self.left.diagonal
        x = self.left.x + math.ceil(offset/2)
        y = self.left.y + math.floor(offset/2)
        size = self.size if offset %2 else self.size + 1
        return Dline(x, y, size)



In [231]:
def draw(d, size=(21,21)):
    A = np.full(size, '.')
    for l in range(A.shape[0] + A.shape[1]):
        for p in d.line_on(l):
            A[p.x, p.y] = str(l % 10)
    A[d.x, d.y] = 'S'
    A[d.left.x, d.left.y] = 'L'
    A[d.top.x, d.top.y] = 'T'
    A[d.bottom.x, d.bottom.y] = 'B'
    A[d.right.x, d.right.y] = 'R'
    print(*[''.join(r) for r in A.T], sep='\n')

In [232]:
d = Diamond(10,10,8)
draw(d)

.....................
.....................
..........T..........
.........234.........
........23456........
.......2345678.......
......234567890......
.....23456789012.....
....2345678901234....
...234567890123456...
..L3456789S1234567R..
...456789012345678...
....6789012345678....
.....89012345678.....
......012345678......
.......2345678.......
........45678........
.........678.........
..........B..........
.....................
.....................


In [233]:
def get_changepoints(diamonds):
    return sorted({d.bottom.diagonal+1 for d in diamonds})

In [236]:
def find_in(lines, mx=1000):
    diamonds = [Diamond.from_sensorbeacon(s,b) for (s,b) in lines]
    changepoints = get_changepoints(diamonds)

    holes = set()
    for diag in changepoints:
        sd = [diamond.line_on(diag) for diamond in diamonds]
        sd = [d for d in sd if d.length > 0]
        sd = sorted(sd, key=lambda d: d.x)

        if len(sd) > 0:
            xe = sd[0].x + sd[0].length
            for dl in sd:
                if dl.x > xe:
                    holes.add((xe, diag -xe))
                xe = max(dl.x + dl.length, xe)
             
    return holes.pop()

In [240]:
(x,y) = find_in(read('test'), mx=20)
print(x,y)
print(int(x*4000000 + y))

14 11
56000011


In [243]:
(x,y) = find_in(lines, mx=4000000)
print((x,y))
print(int(x*4000000 + y))


(2655411, 3166538)
10621647166538


In [264]:

def find_in_two(lines, mx=1000):
    diamonds = [Diamond.from_sensorbeacon(s,b) for (s,b) in lines]
    ch = {pt for d in diamonds for pt in d.corners}

    for p1 in list(ch):
        for p2 in list(ch): 
            ch.add(point.from_do(p1.diagonal, p2.offset))
            ch.add(point.from_do(p2.diagonal, p1.offset))

    for pt in list(ch):
        for dx in [-1,0,1]:
            for dy in [-1,0,1]:
                ch.add(point(pt.x+dx, pt.y+dy))



    for pt in ch:
        if not any(dia.contains(pt) for dia in diamonds):
            if pt.x >= 0 and pt.y >= 0 and pt.x <= mx and pt.y <= mx:
                return pt


In [265]:
pt = find_in_two(read('test'), mx=20)
print(pt)
print(int(pt.x*4000000 + pt.y))

point(x=14.0, y=11.0)
56000011


In [266]:
pt = find_in_two(read(), mx=4000000)
print(pt)
print(int(pt.x*4000000 + pt.y))

point(x=2655411.0, y=3166538.0)
10621647166538
